In [2]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

Compiling /workdir/Main.sc

Downloaded https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3_2.12/maven-metadata.xml
Downloaded https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3_2.12/3.4.4/chisel3_2.12-3.4.4.pom
Downloaded https://repo1.maven.org/maven2/org/scala-lang/scala-library/2.12.12/scala-library-2.12.12.pom
Downloaded https://repo1.maven.org/maven2/org/scala-lang/scala-reflect/2.12.12/scala-reflect-2.12.12.pom
Downloaded https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3-core_2.12/3.4.4/chisel3-core_2.12-3.4.4.pom
Downloaded https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3-macros_2.12/3.4.4/chisel3-macros_2.12-3.4.4.pom
Downloaded https://repo1.maven.org/maven2/edu/berkeley/cs/firrtl_2.12/1.4.4/firrtl_2.12-1.4.4.pom
Downloaded https://repo1.maven.org/maven2/org/scala-lang/scala-reflect/2.12.13/scala-reflect-2.12.13.pom
Downloaded https://repo1.maven.org/maven2/org/scala-lang/scala-library/2.12.13/scala-library-2.12.13.pom
Downloaded https://repo1.maven.org/maven2/com/google/protobuf/protob

Compiling /workdir/Main.sc #2

path: String = "/workdir/source/load-ivy.sc"

In [3]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

import chisel3._

import chisel3.util._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

<h1 style="text-align: center;">Chisel4ml: Generating Fast Implementations of Deeply Quantized Neural Networks using Chisel Generators</h1>
<div style="text-align: center;">
    <img src="slike/e7-logo.png" width=30%>
</div>


<h3 style="text-align: center;">Jure Vreča</h3>
<h4 style="text-align: center;">jure.vreca@ijs.si</h4>

# Chisel
* Constructing Hardware in a Scala Embedded Language
* Chisel is not HLS.
* Type-Safe Meta-Programming for RTL in Scala:
    * Parametrized types
    * Object-oriented programming
    * Functional programming
    * Static type checking

Note: Some of the slides and material were taken from: https://github.com/freechipsproject/chisel-bootcamp

# Example
<div style="text-align: center;">
    <img src="slike/FIR_diagram.png" width=75%>
</div>

In [4]:
class MovingAverage3(bitWidth: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(bitWidth.W))
    val out = Output(UInt(bitWidth.W))
  })
  val z1 = RegNext(io.in) // Create a register whose input is connected to the argument io.in
  val z2 = RegNext(z1)    // Create a register whose input is connected to the argument z1
  io.out := (io.in * 1.U) + (z1 * 1.U) + (z2 * 1.U) // `1.U` is an unsigned literal with value 1
}

defined class MovingAverage3

In [5]:
visualize(() => new MovingAverage3(8))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<?xml-stylesheet href="styles.css" type="text/css"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.43.0 (0)
 -->
<!-- Title: MovingAverage3 Pages: 1 -->
 
 
 MovingAverage3 
 
 
 cluster_MovingAverage3 
 
 MovingAverage3 
 
<!-- cluster_MovingAverage3_clock -->
 
 cluster_MovingAverage3_clock 
 
 clock 
 
<!-- cluster_MovingAverage3_reset -->
 
 cluster_MovingAverage3_reset 
 
 reset 
 
<!-- cluster_MovingAverage3_io_in -->
 
 cluster_MovingAverage3_io_in 
 
 io_in 
 
<!-- struct_cluster_MovingAverage3_z1 -->
 
 struct_cluster_MovingAverage3_z1 
 
 
 ● 
 
 z1 
 
 ● 
 
<!-- cluster_MovingAverage3_io_in->struct_cluster_MovingAverage3_z1 -->
 
 cluster_MovingAverage3_io_in->struct_cluster_MovingAverage3_z1:in 
 
 
 
<!-- op_mul_1 -->
 
 op_mul_1 
 
 
 ● 
 
 mul 
 
 ● 
 
 1 
 
<!-- cluster_MovingAverage3_io_in->op_mul_1 -->
 
 cluster_MovingAverage3_io_in->op_mul_1:in1 
 
 
 
<!-- cluster_MovingAverage3_io_out -->
 
 cluster_MovingAverage3_io_out 
 
 io_out 
 
<!-- struct_cluster_MovingAverage3_z2 -->
 
 struct_cluster_MovingAverage3_z2 
 
 
 ● 
 
 z2 
 
 ● 
 
<!-- struct_cluster_MovingAverage3_z1->struct_cluster_MovingAverage3_z2 -->
 
 struct_cluster_MovingAverage3_z1:out->struct_cluster_MovingAverage3_z2:in 
 
 
 
<!-- op_mul_7 -->
 
 op_mul_7 
 
 
 ● 
 
 mul 
 
 ● 
 
 1 
 
<!-- struct_cluster_MovingAverage3_z1->op_mul_7 -->
 
 struct_cluster_MovingAverage3_z1:out->op_mul_7:in1 
 
 
 
<!-- op_mul_8 -->
 
 op_mul_8 
 
 
 ● 
 
 mul 
 
 ● 
 
 1 
 
<!-- struct_cluster_MovingAverage3_z2->op_mul_8 -->
 
 struct_cluster_MovingAverage3_z2:out->op_mul_8:in1 
 
 
 
<!-- cluster_MovingAverage3__T -->
 
 cluster_MovingAverage3__T 
 
 _T 
 
<!-- op_add_6 -->
 
 op_add_6 
 
 
 ● 
 
 add 
 
 ● 
 
 ● 
 
<!-- cluster_MovingAverage3__T->op_add_6 -->
 
 cluster_MovingAverage3__T->op_add_6:in1 
 
 
 
<!-- op_mul_1->cluster_MovingAverage3__T -->
 
 op_mul_1:out->cluster_MovingAverage3__T 
 
 
 
<!-- op_bits_2 -->
 
 op_bits_2 
 
 
 ● 
 
 bits 
 
 ● 
 
 (7, 0) 
 
<!-- op_bits_2->cluster_MovingAverage3_io_out -->
 
 op_bits_2:out->cluster_MovingAverage3_io_out 
 
 
 
<!-- op_tail_3 -->
 
 op_tail_3 
 
 
 ● 
 
 tail 
 
 ● 
 
 1 
 
<!-- op_tail_3->op_bits_2 -->
 
 op_tail_3:out->op_bits_2:in1 
 
 
 
<!-- op_add_4 -->
 
 op_add_4 
 
 
 ● 
 
 add 
 
 ● 
 
 ● 
 
<!-- op_add_4->op_tail_3 -->
 
 op_add_4:out->op_tail_3:in1 
 
 
 
<!-- op_tail_5 -->
 
 op_tail_5 
 
 
 ● 
 
 tail 
 
 ● 
 
 1 
 
<!-- op_tail_5->op_add_4 -->
 
 op_tail_5:out->op_add_4:in1 
 
 
 
<!-- op_add_6->op_tail_5 -->
 
 op_add_6:out->op_tail_5:in1 
 
 
 
<!-- op_mul_7->op_add_6 -->
 
 op_mul_7:out->op_add_6:in2 
 
 
 
<!-- op_mul_8->op_add_4 -->
 
 op_mul_8:out->op_add_4:in2

In [6]:
//print(getFirrtl(new MovingAverage3(8)))
print(getVerilog(new MovingAverage3(8)))

Elaborating design...
Done elaborating.
module MovingAverage3(
  input        clock,
  input        reset,
  input  [7:0] io_in,
  output [7:0] io_out
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
  reg [31:0] _RAND_1;
`endif // RANDOMIZE_REG_INIT
  reg [7:0] z1; // @[cmd3.sc 6:19]
  reg [7:0] z2; // @[cmd3.sc 7:19]
  wire [8:0] _T = io_in * 1'h1; // @[cmd3.sc 8:20]
  wire [8:0] _T_1 = z1 * 1'h1; // @[cmd3.sc 8:33]
  wire [8:0] _T_3 = _T + _T_1; // @[cmd3.sc 8:27]
  wire [8:0] _T_4 = z2 * 1'h1; // @[cmd3.sc 8:46]
  wire [8:0] _T_6 = _T_3 + _T_4; // @[cmd3.sc 8:40]
  assign io_out = _T_6[7:0]; // @[cmd3.sc 8:10]
  always @(posedge clock) begin
    z1 <= io_in; // @[cmd3.sc 6:19]
    z2 <= z1; // @[cmd3.sc 7:19]
  end
// Register and memory initialization
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM


this are my notes. bla bla

# FIR Generator

In [ ]:
// Generalized FIR filter parameterized by the convolution coefficients
class FirFilter(bitWidth: Int, coeffs: Seq[UInt]) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(bitWidth.W))
    val out = Output(UInt())
  })
  // Create the serial-in, parallel-out shift register
  val zs = Reg(Vec(coeffs.length, UInt(bitWidth.W)))
  zs(0) := io.in
  for (i <- 1 until coeffs.length) {
    zs(i) := zs(i-1)
  }

  // Do the multiplies
  val products = VecInit.tabulate(coeffs.length)(i => zs(i) * coeffs(i))

  // Sum up the products
  io.out := products.reduce(_ +& _)
}

In [ ]:
// same 3-point moving average filter as before
visualize(() => new FirFilter(8, Seq(1.U, 1.U, 1.U)))

// 1-cycle delay as a FIR filter
//visualize(() => new FirFilter(8, Seq(0.U, 1.U)))

// 5-point FIR filter with a triangle impulse response
//visualize(() => new FirFilter(8, Seq(1.U, 2.U, 3.U, 2.U, 1.U)))

# Artificial Neural Networks
<div style="text-align: center;">
    <img src="slike/neuron.drawio.png" width=45%>
</div>
<h1 style="text-align: center;">
$$
\displaystyle y=f(b + \sum_{i=0}^{N-1} x_i \cdot w_i)
$$
</h1>


# How does chisel4ml use Chisel?

In [7]:
object Neuron {
    def apply[I <: Bits,
              W <: Bits,
              M <: Bits,
              A <: Bits,
              O <: Bits](in: Seq[I],
                         weights: Seq[W],
                         thresh: A,
                         mul: (I, W) => M,
                         add: Vec[M] => A,
                         actFn: (A, A) => O,
                         shift: Int): O = {
        val muls = VecInit((in zip weights).map{
            case (a,b) => mul(a,b)
        })
        val pAct = add(muls)
        val sAct = (pAct << shift.abs).asTypeOf(pAct)
        actFn(sAct, thresh)
    }
}

defined object Neuron

In [8]:
def mulUQ(i: SInt, w: SInt): SInt = i * w    // Uniform quantization
def addUQ = (x: Vec[SInt]) => x.reduceTree(_ +& _)

def mulBW = (i: SInt, w: Bool) => Mux(w, i, -i)  // Binary weight quantization

def mulBNN(i: Bool, w: Bool): Bool = ~(i ^ w) // Binarized quantization
def addBNN = (x: Vec[Bool]) => PopCount(x.asUInt)

def reluFn(act: SInt, thresh: SInt): UInt = Mux((act - thresh) > 0.S, (act - thresh).asUInt, 0.U)
def signFn(act:UInt, thresh: UInt): Bool = act >= thresh

defined function mulUQ
defined function addUQ
defined function mulBW
defined function mulBNN
defined function addBNN
defined function reluFn
defined function signFn

In [9]:
class DummyUniformModule extends Module {
  val io = IO(new Bundle {
    val in = Input(Vec(3, SInt(4.W)))
    val out = Output(UInt())
  })
    io.out := Neuron[SInt, SInt, SInt, SInt, UInt](in = io.in,
                                                   weights = Seq(1.S, -2.S, 3.S),
                                                   thresh = -1.S,
                                                   mul = mulUQ,
                                                   add = addUQ,
                                                   actFn = reluFn,
                                                   shift = 1
                                                 )
}

defined class DummyUniformModule

In [10]:
    visualize(() => new DummyUniformModule())
//print(getFirrtl(new DummyUniformModule()))
//print(getVerilog(new DummyUniformModule()))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<?xml-stylesheet href="styles.css" type="text/css"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.43.0 (0)
 -->
<!-- Title: DummyUniformModule Pages: 1 -->
 
 
 DummyUniformModule 
 
 
 cluster_DummyUniformModule 
 
 DummyUniformModule 
 
<!-- cluster_DummyUniformModule_clock -->
 
 cluster_DummyUniformModule_clock 
 
 clock 
 
<!-- cluster_DummyUniformModule_reset -->
 
 cluster_DummyUniformModule_reset 
 
 reset 
 
<!-- cluster_DummyUniformModule_io_in_0 -->
 
 cluster_DummyUniformModule_io_in_0 
 
 io_in_0 
 
<!-- op_mul_9 -->
 
 op_mul_9 
 
 
 ● 
 
 mul 
 
 ● 
 
 1 
 
<!-- cluster_DummyUniformModule_io_in_0->op_mul_9 -->
 
 cluster_DummyUniformModule_io_in_0->op_mul_9:in1 
 
 
 
<!-- cluster_DummyUniformModule_io_in_1 -->
 
 cluster_DummyUniformModule_io_in_1 
 
 io_in_1 
 
<!-- op_mul_12 -->
 
 op_mul_12 
 
 
 ● 
 
 mul 
 
 ● 
 
 -2 
 
<!-- cluster_DummyUniformModule_io_in_1->op_mul_12 -->
 
 cluster_DummyUniformModule_io_in_1->op_mul_12:in1 
 
 
 
<!-- cluster_DummyUniformModule_io_in_2 -->
 
 cluster_DummyUniformModule_io_in_2 
 
 io_in_2 
 
<!-- op_mul_14 -->
 
 op_mul_14 
 
 
 ● 
 
 mul 
 
 ● 
 
 3 
 
<!-- cluster_DummyUniformModule_io_in_2->op_mul_14 -->
 
 cluster_DummyUniformModule_io_in_2->op_mul_14:in1 
 
 
 
<!-- cluster_DummyUniformModule_io_out -->
 
 cluster_DummyUniformModule_io_out 
 
 io_out 
 
<!-- cluster_DummyUniformModule__T -->
 
 cluster_DummyUniformModule__T 
 
 _T 
 
<!-- op_pad_10 -->
 
 op_pad_10 
 
 
 ● 
 
 pad 
 
 ● 
 
<!-- cluster_DummyUniformModule__T->op_pad_10 -->
 
 cluster_DummyUniformModule__T->op_pad_10:in1 
 
 
 
<!-- op_mul_9->cluster_DummyUniformModule__T -->
 
 op_mul_9:out->cluster_DummyUniformModule__T 
 
 
 
<!-- cluster_DummyUniformModule__WIRE__0 -->
 
 cluster_DummyUniformModule__WIRE__0 
 
 _WIRE__0 
 
<!-- op_add_21 -->
 
 op_add_21 
 
 
 ● 
 
 add 
 
 ● 
 
 ● 
 
<!-- cluster_DummyUniformModule__WIRE__0->op_add_21 -->
 
 cluster_DummyUniformModule__WIRE__0->op_add_21:in1 
 
 
 
<!-- op_pad_10->cluster_DummyUniformModule__WIRE__0 -->
 
 op_pad_10:out->cluster_DummyUniformModule__WIRE__0 
 
 
 
<!-- cluster_DummyUniformModule__WIRE__1 -->
 
 cluster_DummyUniformModule__WIRE__1 
 
 _WIRE__1 
 
<!-- cluster_DummyUniformModule__WIRE__1->op_add_21 -->
 
 cluster_DummyUniformModule__WIRE__1->op_add_21:in2 
 
 
 
<!-- op_pad_11 -->
 
 op_pad_11 
 
 
 ● 
 
 pad 
 
 ● 
 
<!-- op_pad_11->cluster_DummyUniformModule__WIRE__1 -->
 
 op_pad_11:out->cluster_DummyUniformModule__WIRE__1 
 
 
 
<!-- op_mul_12->op_pad_11 -->
 
 op_mul_12:out->op_pad_11:in1 
 
 
 
<!-- cluster_DummyUniformModule__WIRE_1_1 -->
 
 cluster_DummyUniformModule__WIRE_1_1 
 
 _WIRE_1_1 
 
<!-- op_add_20 -->
 
 op_add_20 
 
 
 ● 
 
 add 
 
 ● 
 
 ● 
 
<!-- cluster_DummyUniformModule__WIRE_1_1->op_add_20 -->
 
 cluster_DummyUniformModule__WIRE_1_1->op_add_20:in2 
 
 
 
<!-- op_pad_13 -->
 
 op_pad_13 
 
 
 ● 
 
 pad 
 
 ● 
 
<!-- op_pad_13->cluster_DummyUniformModule__WIRE_1_1 -->
 
 op_pad_13:out->cluster_DummyUniformModule__WIRE_1_1 
 
 
 
<!-- op_mul_14->op_pad_13 -->
 
 op_mul_14:out->op_pad_13:in1 
 
 
 
<!-- cluster_DummyUniformModule__WIRE_3 -->
 
 cluster_DummyUniformModule__WIRE_3 
 
 _WIRE_3 
 
<!-- op_sub_25 -->
 
 op_sub_25 
 
 
 ● 
 
 sub 
 
 ● 
 
 -1 
 
<!-- cluster_DummyUniformModule__WIRE_3->op_sub_25 -->
 
 cluster_DummyUniformModule__WIRE_3->op_sub_25:in1 
 
 
 
<!-- op_sub_29 -->
 
 op_sub_29 
 
 
 ● 
 
 sub 
 
 ● 
 
 -1 
 
<!-- cluster_DummyUniformModule__WIRE_3->op_sub_29 -->
 
 cluster_DummyUniformModule__WIRE_3->op_sub_29:in1 
 
 
 
<!-- op_asSInt_15 -->
 
 op_asSInt_15 
 
 
 ● 
 
 asSInt 
 
 ● 
 
<!-- op_asSInt_15->cluster_DummyUniformModule__WIRE_3 -->
 
 op_asSInt_15:out->cluster_DummyUniformModule__WIRE_3 
 
 
 
<!-- op_bits_16 -->
 
 op_bits_16 
 
 
 ● 
 
 bits 
 
 ● 
 
 (8, 0) 
 
<!-- op_bits_16->op_asSInt_15 -->
 
 op_bits_16:out->op_asSInt_15:in1

In [11]:
class DummyBinarizedModule extends Module {
  val io = IO(new Bundle {
    val in = Input(Vec(3, Bool()))
    val out = Output(UInt())
  })
    io.out := Neuron[Bool, Bool, Bool, UInt, Bool](in = io.in,
                                                  weights = Seq(true.B, false.B, true.B),
                                                  thresh = 2.U,
                                                  mul = mulBNN,
                                                  add = addBNN,
                                                  actFn = signFn,
                                                  shift = 0
                                                 )
}

defined class DummyBinarizedModule

In [12]:
visualize(() => new DummyBinarizedModule())
print(getFirrtl(new DummyBinarizedModule()))
//print(getVerilog(new DummyBinarizedModule()))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<?xml-stylesheet href="styles.css" type="text/css"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.43.0 (0)
 -->
<!-- Title: DummyBinarizedModule Pages: 1 -->
 
 
 DummyBinarizedModule 
 
 
 cluster_DummyBinarizedModule 
 
 DummyBinarizedModule 
 
<!-- cluster_DummyBinarizedModule_clock -->
 
 cluster_DummyBinarizedModule_clock 
 
 clock 
 
<!-- cluster_DummyBinarizedModule_reset -->
 
 cluster_DummyBinarizedModule_reset 
 
 reset 
 
<!-- cluster_DummyBinarizedModule_io_in_0 -->
 
 cluster_DummyBinarizedModule_io_in_0 
 
 io_in_0 
 
<!-- op_xor_30 -->
 
 op_xor_30 
 
 
 ● 
 
 xor 
 
 ● 
 
 1 
 
<!-- cluster_DummyBinarizedModule_io_in_0->op_xor_30 -->
 
 cluster_DummyBinarizedModule_io_in_0->op_xor_30:in1 
 
 
 
<!-- cluster_DummyBinarizedModule_io_in_1 -->
 
 cluster_DummyBinarizedModule_io_in_1 
 
 io_in_1 
 
<!-- op_not_40 -->
 
 op_not_40 
 
 
 ● 
 
 not 
 
 ● 
 
<!-- cluster_DummyBinarizedModule_io_in_1->op_not_40 -->
 
 cluster_DummyBinarizedModule_io_in_1->op_not_40:in1 
 
 
 
<!-- op_not_48 -->
 
 op_not_48 
 
 
 ● 
 
 not 
 
 ● 
 
<!-- cluster_DummyBinarizedModule_io_in_1->op_not_48 -->
 
 cluster_DummyBinarizedModule_io_in_1->op_not_48:in1 
 
 
 
<!-- op_not_55 -->
 
 op_not_55 
 
 
 ● 
 
 not 
 
 ● 
 
<!-- cluster_DummyBinarizedModule_io_in_1->op_not_55 -->
 
 cluster_DummyBinarizedModule_io_in_1->op_not_55:in1 
 
 
 
<!-- cluster_DummyBinarizedModule_io_in_2 -->
 
 cluster_DummyBinarizedModule_io_in_2 
 
 io_in_2 
 
<!-- op_xor_39 -->
 
 op_xor_39 
 
 
 ● 
 
 xor 
 
 ● 
 
 1 
 
<!-- cluster_DummyBinarizedModule_io_in_2->op_xor_39 -->
 
 cluster_DummyBinarizedModule_io_in_2->op_xor_39:in1 
 
 
 
<!-- op_xor_47 -->
 
 op_xor_47 
 
 
 ● 
 
 xor 
 
 ● 
 
 1 
 
<!-- cluster_DummyBinarizedModule_io_in_2->op_xor_47 -->
 
 cluster_DummyBinarizedModule_io_in_2->op_xor_47:in1 
 
 
 
<!-- op_xor_54 -->
 
 op_xor_54 
 
 
 ● 
 
 xor 
 
 ● 
 
 1 
 
<!-- cluster_DummyBinarizedModule_io_in_2->op_xor_54 -->
 
 cluster_DummyBinarizedModule_io_in_2->op_xor_54:in1 
 
 
 
<!-- cluster_DummyBinarizedModule_io_out -->
 
 cluster_DummyBinarizedModule_io_out 
 
 io_out 
 
<!-- cluster_DummyBinarizedModule__T -->
 
 cluster_DummyBinarizedModule__T 
 
 _T 
 
<!-- op_not_41 -->
 
 op_not_41 
 
 
 ● 
 
 not 
 
 ● 
 
<!-- cluster_DummyBinarizedModule__T->op_not_41 -->
 
 cluster_DummyBinarizedModule__T->op_not_41:in1 
 
 
 
<!-- op_not_49 -->
 
 op_not_49 
 
 
 ● 
 
 not 
 
 ● 
 
<!-- cluster_DummyBinarizedModule__T->op_not_49 -->
 
 cluster_DummyBinarizedModule__T->op_not_49:in1 
 
 
 
<!-- op_not_56 -->
 
 op_not_56 
 
 
 ● 
 
 not 
 
 ● 
 
<!-- cluster_DummyBinarizedModule__T->op_not_56 -->
 
 cluster_DummyBinarizedModule__T->op_not_56:in1 
 
 
 
<!-- op_xor_30->cluster_DummyBinarizedModule__T -->
 
 op_xor_30:out->cluster_DummyBinarizedModule__T 
 
 
 
<!-- op_gte_31 -->
 
 op_gte_31 
 
 
 ● 
 
 gte 
 
 ● 
 
 2 
 
<!-- op_gte_31->cluster_DummyBinarizedModule_io_out -->
 
 op_gte_31:out->cluster_DummyBinarizedModule_io_out 
 
 
 
<!-- op_bits_32 -->
 
 op_bits_32 
 
 
 ● 
 
 bits 
 
 ● 
 
 (1, 0) 
 
<!-- op_bits_32->op_gte_31 -->
 
 op_bits_32:out->op_gte_31:in1 
 
 
 
<!-- op_add_33 -->
 
 op_add_33 
 
 
 ● 
 
 add 
 
 ● 
 
 ● 
 
<!-- op_add_33->op_bits_32 -->
 
 op_add_33:out->op_bits_32:in1 
 
 
 
<!-- op_pad_34 -->
 
 op_pad_34 
 
 
 ● 
 
 pad 
 
 ● 
 
<!-- op_pad_34->op_add_33 -->
 
 op_pad_34:out->op_add_33:in1 
 
 
 
<!-- op_bits_35 -->
 
 op_bits_35 
 
 
 ● 
 
 bits 
 
 ● 
 
 (0, 0) 
 
<!-- op_bits_35->op_pad_34 -->
 
 op_bits_35:out->op_pad_34:in1 
 
 
 
<!-- op_cat_36 -->
 
 op_cat_36 
 
 
 ● 
 
 cat 
 
 ● 
 
 ● 
 
<!-- op_cat_36->op_bits_35 -->
 
 op_cat_36:out->op_bits_35:in1 
 
 
 
<!-- op_cat_37 -->
 
 op_cat_37 
 
 
 ● 
 
 cat 
 
 ● 
 
 ● 
 
<!-- op_cat_37->op_cat_36 -->
 
 op_cat_37:out->op_cat_36:in1 
 
 
 
<!-- op_not_38 -->
 
 op_not_38 
 
 
 ● 
 
 not 
 
 ● 
 
<!-- op_not_38->op_

circuit DummyBinarizedModule :
  module DummyBinarizedModule :
    input clock : Clock
    input reset : UInt<1>
    output io : { flip in : UInt<1>[3], out : UInt}

    node _T = xor(io.in[0], UInt<1>("h1")) @[cmd7.sc 6:42]
    node _T_1 = not(_T) @[cmd7.sc 6:38]
    node _T_2 = xor(io.in[1], UInt<1>("h0")) @[cmd7.sc 6:42]
    node _T_3 = not(_T_2) @[cmd7.sc 6:38]
    node _T_4 = xor(io.in[2], UInt<1>("h1")) @[cmd7.sc 6:42]
    node _T_5 = not(_T_4) @[cmd7.sc 6:38]
    wire _WIRE : UInt<1>[3] @[cmd6.sc 13:27]
    _WIRE[0] <= _T_1 @[cmd6.sc 13:27]
    _WIRE[1] <= _T_3 @[cmd6.sc 13:27]
    _WIRE[2] <= _T_5 @[cmd6.sc 13:27]
    node hi = cat(_WIRE[2], _WIRE[1]) @[cmd7.sc 7:43]
    node _T_6 = cat(hi, _WIRE[0]) @[cmd7.sc 7:43]
    node _T_7 = bits(_T_6, 0, 0) @[Bitwise.scala 49:65]
    node _T_8 = bits(_T_6, 1, 1) @[Bitwise.scala 49:65]
    node _T_9 = bits(_T_6, 2, 2) @[Bitwise.scala 49:65]
    node _T_10 = add(_T_8, _T_9) @[Bitwise.scala 47:55]
    node _T_11 = bits(_T_10, 1, 0) @[Bitwi

## Other abstractions in Chisel4ml:
* ProcessingElement == layer
* ProcessingPipeline == model

<p align="center">
<img src="slike/ProcElementC4ml.png" width=450  height=400 align=left>
<img src="slike/ProcPipelineC4ml.png" width=450 height=400 align=right>
</p>